In [40]:
import cv2
import numpy as np
import joblib
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [41]:
CLASS_MAPPING = {
    "cardboard": 0,
    "glass": 1,
    "metal": 2,
    "paper": 3,
    "plastic": 4,
    "trash": 5,
    "unknown": 6
}

## Load CNN Model

In [42]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
x = GlobalAveragePooling2D()(base_model.output)
cnn_model = Model(base_model.input, x)


## Load Pipeline Components

In [43]:
scaler = joblib.load("../models/scaler.pkl")
classifier = joblib.load("../models/svm_model.pkl")  # or knn_model.pkl


In [44]:
CLASS_NAMES = classifier.classes_
UNKNOWN_THRESHOLD = 0.65

## Preprocess Frame

In [45]:
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = np.expand_dims(frame, axis=0)
    return preprocess_input(frame)

## ROI Size

In [46]:
ROI_SIZE = 250

## Live Camera App

In [47]:
cap = cv2.VideoCapture(0)
print("Place the material inside the box. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    cx, cy = w // 2, h // 2
    half = ROI_SIZE // 2

    # ROI coordinates
    x1, y1 = cx - half, cy - half
    x2, y2 = cx + half, cy + half

    # Draw ROI box
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(frame, "Place material inside the box",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                (0, 255, 0), 2)

    # Crop ROI safely
    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        cv2.imshow("Material Classification", frame)
        continue

    # Feature extraction
    img = preprocess_frame(roi)
    features = cnn_model.predict(img, verbose=0)
    features = scaler.transform(features)

    # Prediction with rejection
    probs = classifier.predict_proba(features)[0]
    max_prob = np.max(probs)

    if max_prob < UNKNOWN_THRESHOLD:
        label = "unknown"
        labelNumber = CLASS_MAPPING.get(label)
        confidence = max_prob
        color = (0, 0, 255)
    else:
        label = CLASS_NAMES[np.argmax(probs)]
        labelNumber = CLASS_MAPPING.get(label)
        confidence = max_prob
        color = (0, 255, 0)

    # Display prediction
    text = f"{label} --> {labelNumber} ({confidence:.2f})"
    cv2.putText(frame, text,
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                color, 2)

    cv2.imshow("Material Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Place the material inside the box. Press 'q' to quit.
